In [2]:
from scaling_and_reproject import get_crs, compare_crs,reproject_data,mask_raster_data,Min_Max_Scaling,Z_score_scaling
import numpy as np
import numpy.ma as ma
import geopandas as gpd
import rasterio
import rioxarray as rxr

In [9]:
#example usage
# Load vector data
vector = gpd.read_file("/mnt/CA6A062B6A06153B/study/Geospatial processing/gp-temporary/test/Municipalities_on_lakes.shp")
# Open raster
raster = rxr.open_rasterio("/mnt/CA6A062B6A06153B/study/Geospatial processing/gp-temporary/test/l8monzaali_without_csr.tif")# ,masked=True)

In [10]:
#get crs
raster_crs=get_crs(raster)
vector_crs=get_crs(vector)
print(raster_crs)
print(vector_crs)

None
EPSG:32632


In [11]:
#compare crs
result = compare_crs(raster, vector)
print(result)

{'raster_crs': 'No CRS defined', 'vector_crs': 'EPSG:32632', 'same_crs': False, 'error': None}


In [12]:
# Reproject
reprojected_vector = reproject_data(vector, "EPSG:4326")
reprojected_raster = reproject_data(raster, "EPSG:3857")

# Save output (maintains all geospatial information)
reprojected_raster.rio.to_raster("/mnt/CA6A062B6A06153B/study/Geospatial processing/gp-temporary/test/output.tif")

print(get_crs(reprojected_raster))
print(get_crs(reprojected_vector))

ValueError: Raster data has no CRS. Cannot reproject.

In [ ]:
#mask no value data
#for raster loded by rioxarray
data = rxr.open_rasterio("/mnt/CA6A062B6A06153B/study/Geospatial processing/gp-temporary/test/output.tif")
masked_data, mask = mask_raster_data(data, return_mask=True)
print("masked_data:",masked_data)
print("mask:",mask)


#Undefined No-Data: If the raster lacks nodata metadata, specify it manually:
# Specify a no-data value explicitly (e.g., -9999)
no_data_value = -9999
masked_data, mask = mask_raster_data(data, no_data_value=no_data_value, return_mask=True)


#Multi-Band Masks: To mask pixels where any band has no-data:
combined_mask = mask.all(dim="band")  # Mask where all bands are valid
masked_data = data.where(combined_mask)
# Flatten the data for scikit-learn
valid_data = masked_data.stack(samples=("y", "x")).dropna(dim="samples")
print("valid_data:",valid_data)

In [ ]:
#for raster loaded by rasterio
with rasterio.open("/mnt/CA6A062B6A06153B/study/Geospatial processing/gp-temporary/test/output.tif") as src:
    data = src.read()
    profile = src.profile
masked_data, mask, profile = mask_raster_data(data, profile=profile, return_mask=True)
print("maksed_data:",masked_data)
print("mask:",mask)

#If the raster lacks nodata metadata,specify a no-data value explicitly (e.g., -9999)
no_data_value = -9999
# Mask the raster data
masked_data, mask, profile = mask_raster_data(data, no_data_value=no_data_value, return_mask=True)



In [ ]:
# Create a combined mask for all bands (valid if all bands are valid)
multi_band_mask = np.all(mask, axis=0)  # Shape: [height, width]
masked_data = ma.masked_array(data, mask=np.broadcast_to(~mask, data.shape))

# For classification, extract valid pixels (across all bands)
valid_samples = masked_data.compressed()  # Flattened array of valid values
print("valid_samples:",valid_samples)

In [ ]:
#z_ score scaling
raster=Z_score_scaling(raster)

In [ ]:
#min_max scaling
raster=Min_Max_Scaling(raster)

In [8]:
import rasterio
from rasterio.shutil import copy

# Open the input file
with rasterio.open("/mnt/CA6A062B6A06153B/study/Geospatial processing/test/l8monzaali.tif") as src:
    # Copy the dataset while omitting the CRS
    profile = src.profile.copy()
    profile.pop("crs", None)  # Remove the CRS

    with rasterio.open("l8monzaali_without_csr.tif", "w", **profile) as dst:
        dst.write(src.read())  # Copy data
